In [1]:
import torch
import numpy as np

In [2]:
step = 10e-11
spectrum_s = 300e-9
spectrum_e = 800e-9
c = 3e8 
lambda_0 = 550e-9
alpha = 0.1
wavelength_delta = 25e-9
wavelength_step = round(wavelength_delta / step)
num_wavelengths = [3, 6, 12]
sigma_arr = [0.9e14, 2e14, 4.7e14]

a = 12 * 10e-6
n_steps = 1000
start=-140e-6
end=140e-6
step_m = (end - start) / n_steps
range_ = torch.linspace(start, end, steps=n_steps)
x_range = range_.unsqueeze(0).t()
y_range = range_.unsqueeze(0)

In [3]:
z_range = [0.15e-2, 0.5e-2, 1e-2, 1.55e-2, 5.5e-2]
x_1 = 140e-6 
y_1 = 140e-6
coherence_length_arr_3 = []
coherence_length_arr_6 = []
coherence_length_arr_12 = []
for n, sigma in zip(num_wavelengths, sigma_arr):
    wavelengths_intensity = []
    wavelengths_val = []
    lambda_arr = torch.arange(spectrum_s, spectrum_e, step)
    const_1 = ((2 * np.pi * c) / (lambda_0))**2
    const_2 = 2 * sigma ** 2
    function_in_lambda = const_1 * (((lambda_0 - lambda_arr) / lambda_arr) ** 2) / const_2
    exp_lambda = torch.exp(-1 * function_in_lambda)
    max_exp_lambda = exp_lambda[torch.argmax(exp_lambda)]
    k = -wavelength_step
    while exp_lambda[len(lambda_arr) // 2 + k] > alpha * max_exp_lambda:
        k -= wavelength_step
    k += wavelength_step
    for i in range(n):
        wavelengths_intensity.append(exp_lambda[len(lambda_arr) // 2 + (k + wavelength_step * i)])
        wavelengths_val.append(lambda_arr[len(lambda_arr) // 2 + (k + wavelength_step * i)])
    wavelengths_intensity = torch.tensor(wavelengths_intensity)
    normalize_intensity = wavelengths_intensity / torch.sum(wavelengths_intensity)
    for z in z_range:
        J_nominator = torch.zeros((x_range.shape[0], x_range.shape[0]), dtype=torch.complex64)
        J_denominator = torch.zeros((x_range.shape[0], x_range.shape[0]), dtype=torch.complex64)
        mu = torch.zeros((x_range.shape[0], x_range.shape[0]), dtype=torch.complex64)
        for wavelength, norm_intens in zip(wavelengths_val, normalize_intensity):
            arg_x = (a * x_range) / (wavelength * z)
            arg_y = (a * y_range) / (wavelength * z)
            psi = (np.pi / (wavelength * z)) * (((x_1+x_range)**2+(y_1+y_range)**2)-(x_1**2+y_1**2))
            J_nominator += (torch.exp(-1j * psi) * (torch.special.sinc(arg_x) * torch.special.sinc(arg_y)) * norm_intens) / (wavelength*z)**2
            J_denominator += norm_intens / (wavelength * z)**2
        mu = J_nominator / J_denominator      
        coherence_area = torch.sum(torch.abs(mu) ** 2) * (step_m**2)
        coherence_length = torch.sqrt(coherence_area)  
        if n==3:
            coherence_length_arr_3.append(coherence_length.item())
        if n==6:
            coherence_length_arr_6.append(coherence_length.item())
        elif n==12:
            coherence_length_arr_12.append(coherence_length.item())

In [4]:
c_l_a_3 = [x * 10**6 for x in coherence_length_arr_3]
c_l_a_6 = [x * 10**6 for x in coherence_length_arr_6]
c_l_a_12 = [x * 10**6 for x in coherence_length_arr_12]

print(f"For x1 {x_1*10**6} \mu m, y1 {y_1*10**6} \mu m")
print("coherence_length_arr_3", c_l_a_3)
print("coherence_length_arr_6", c_l_a_6)
print("coherence_length_arr_12", c_l_a_12)

For x1 140.0 \mu m, y1 140.0 \mu m
coherence_length_arr_3 [6.67482754579396, 22.113437808002345, 43.75648495624773, 66.88325811410323, 204.36935301404446]
coherence_length_arr_6 [6.416744781745365, 21.36202783731278, 42.506017052801326, 65.31525286845863, 202.1526888711378]
coherence_length_arr_12 [5.871146186109399, 19.614724806160666, 39.16808418580331, 60.4235683567822, 191.7638728627935]
